원소를 받은 순서가 아니라 원소간의 상대적인 중요도에 따라 원소를 정렬해야 할 때

우선순위 큐(priority queue)가 적합하다

도서관에서 대출한 책을 관리하는 프로그램을 작성한다고 하면,

회원중에는 계속 신간만 대출하는 사람이 있고, 대출한 책을 제시간에 반납하는 사람이 있고, 연체된 책이 있음을 통지하는 사람이 있다.

대출한 책을 표현하는 클래스이다.



In [1]:
# 만기일을 넘긴 경우에는 연체 사실을 통지하는 메시지를 보내는 시스템이 필요하다.

class Book:
    def __init__(self, title, due_date):
        self.title = title
        self.due_date = due_date
        
def add_book(queue, book):
    queue.append(book)
    queue.sort(key=lambda x: x.due_date, reverse=True) # 날짜 순으로 내림차순 정렬
    
queue = []
add_book(queue, Book('돈키호테', '2020-06-07'))
add_book(queue, Book('프랑켄슈타인', '2020-06-05'))
add_book(queue, Book('레미제라블', '2020-06-08'))
add_book(queue, Book('전쟁과 평화', '2020-06-03'))

In [2]:
class NoOverdueBooks(Exception):
    pass

def next_overdue_book(queue, now) -> Book:
    if queue:
        book = queue[-1]
        if book.due_date < now:
            queue.pop()
            return book
        
    raise NoOverdueBooks

In [4]:
now = '2020-06-10'

found = next_overdue_book(queue, now)
print(found.title)

found = next_overdue_book(queue, now)
print(found.title)

돈키호테
레미제라블


In [5]:
def return_book(queue, book):
    queue.remove(book)
    
queue = []
book = Book('보물섬', '2020-06-04')
 
add_book(queue, book)
print('반납 전:', [x.title for x in queue])

return_book(queue, book)
print('반납 후: ', [x.title for x in queue])

반납 전: ['보물섬']
반납 후:  []


그리고 모든 책이 반납 됬는지 확인 되면 return_book 함수는 정해진 예외를 발생 시킨다. 

None을 반환하기 보다는 예외를 발생시켜라. ^^

In [ ]:
try:
    next_overdue_book(queue, now)
except NoOverdueBooks:
    pass # 이문장이 실행될 것으로 예상됨
else:
    assert False # 이 문장은 결코 실행되지 않음

하지만 이 해결방법의 계산복잡도는 이상적이지 않다

연체된 책을 검사 혹은 제거 하는 비용은 상수 이지만, 책을 추가 할 때마다 전체 리스트를 다시 정렬해야 하는 추가 비용이 들어간다.

추가할 책이 len(queue) 만큼 있다면 이를 정렬하는데 드는 비용은 nlogn이므로

총 비용은 $n^2logn$이다

In [6]:
import random
import timeit

def print_results(count, tests):
    avg_iteration = sum(tests) / len(tests)
    print(f'\n원소 수: {count:>5,} 걸린시간: {avg_iteration:.6f}초')
    return count, avg_iteration


def list_overdue_benchmark(count):
    def prepare():
        to_add = list(range(count))
        random.shuffle(to_add)
        return [], to_add

    def run(queue, to_add):
        for i in to_add:
            queue.append(i)
            queue.sort(reverse=True)

        while queue:
            queue.pop()

    tests = timeit.repeat(
        setup='queue, to_add = prepare()',
        stmt=f'run(queue, to_add)',
        globals=locals(),
        repeat=100,
        number=1)

    return print_results(count, tests)

def print_delta(before, after):
    before_count, before_time = before
    after_count, after_time = after
    growth = 1 + (after_count - before_count) / before_count
    slowdown = 1 + (after_time - before_time) / before_time
    print(f'데이터 크기 {growth:>4.1f}배, 걸린 시간 {slowdown:>4.1f}배')

baseline = list_overdue_benchmark(500)
for count in (1_000, 1_500, 2_000):
    comparison = list_overdue_benchmark(count)
    print_delta(baseline, comparison)


원소 수:   500 걸린시간: 0.003440초

원소 수: 1,000 걸린시간: 0.012054초
데이터 크기  2.0배, 걸린 시간  3.5배

원소 수: 1,500 걸린시간: 0.025646초
데이터 크기  3.0배, 걸린 시간  7.5배

원소 수: 2,000 걸린시간: 0.044412초
데이터 크기  4.0배, 걸린 시간 12.9배


In [7]:

def list_return_benchmark(count):
    def prepare():
        queue = list(range(count))
        random.shuffle(queue)
        to_return = list(range(count))
        random.shuffle(to_return)
        return queue, to_return

    def run(queue, to_return):
        for i in to_return:
            queue.remove(i)

    tests = timeit.repeat(
        setup='queue, to_return = prepare()',
        stmt=f'run(queue, to_return)',
        globals=locals(),
        repeat=100,
        number=1)

    return print_results(count, tests)

baseline = list_return_benchmark(500)
for count in (1_000, 1_500, 2_000):
    comparison = list_return_benchmark(count)
    print_delta(baseline, comparison)


원소 수:   500 걸린시간: 0.002257초

원소 수: 1,000 걸린시간: 0.008641초
데이터 크기  2.0배, 걸린 시간  3.8배

원소 수: 1,500 걸린시간: 0.019097초
데이터 크기  3.0배, 걸린 시간  8.5배

원소 수: 2,000 걸린시간: 0.033810초
데이터 크기  4.0배, 걸린 시간 15.0배


도서관 장서 수가 적은 경우라면 가능한데, 아주 큰 규모의 도서관 같은 경우에는 리스트에 정의된 메서드가 효과적이지 않다.

이러한 문제는 우선순위 큐를 사용해서 해결 가능하다.

heapq에서 힙은 여러 아이템을 유지하되 새로운 원소를 추가하거나, 가장 작은 원소를 제거할 때 로그 복잡도가 드는 데이터 구조이다.

heapq 모듈의 좋은 점은 제대로 작동하는 힙을 어떻게 구현하는지 전혀 신경쓸 필요가 없다는 것이다.


In [8]:
from heapq import heappush

def add_book(queue, book):
    heappush(queue, book)
    

queue = []
add_book(queue, Book('작은 아씨들', '2020-06-05'))
add_book(queue, Book('타임 머신', '2020-05-30')) #  이경우에 에러가 뜬다 why? Book은 Class 객체이므로 무엇을 기준으로 정렬해야할지 모름..

# 이런 경우 어떻게 해야하는가?
# 복잡한 기준을 사용할떄는 key 파라미터를 사용해라

# total_ordering 클래스 데코레이터를 사용하고  __lt__ 특별 메서드를 구현하면ㄷ 된다다

TypeError: '<' not supported between instances of 'Book' and 'Book'

In [9]:
import functools

@functools.total_ordering
class Book:
    def __init__(self, title, due_date):
        self.title = title
        self.due_date = due_date
        
    def __lt__(self, other):
        return self.due_date < other.due_date
    
queue = []
add_book(queue, Book('오만과 편견', '2020-06-01'))
add_book(queue, Book('타임 머신', '2020-05-30'))
add_book(queue, Book('죄와 벌', '2020-06-06'))
add_book(queue, Book('폭풍의 언덕', '2020-06-12'))


In [10]:
# 혹은 모든 책이 들어있는 리스트를 만들고 sort 메서드를 사용해 힙을 만들 수도 있다

queue = [
    Book('오만과 편견', '2020-06-01'),
    Book('타임 머신', '2020-05-30'),
    Book('죄와 벌', '2020-06-06'),
    Book('폭풍의 언덕', '2020-06-12'),
]
queue.sort()

In [11]:
#  혹은 heap.heapify 함수를 사용하면 선형 시간에 힙을 만들 수 있다.
from heapq import heapify

queue = [
    Book('오만과 편견', '2020-06-01'),
    Book('타임 머신', '2020-05-30'),
    Book('죄와 벌', '2020-06-06'),
    Book('폭풍의 언덕', '2020-06-12'),
]
heapify(queue)

In [12]:
from heapq import heappop

def next_overdue_book(queue, now):
    if queue:
        book = queue[0]
        if book.due_date < now: # 만기가 가장 이른 책이 맨 앞에 있다
            heappop(queue) # 연체된 책을 제거
            return book
    raise NoOverdueBooks
            
            
now = '2020-06-02'

book = next_overdue_book(queue, now)
print(book.title)

book = next_overdue_book(queue, now)
print(book.title)

try:
    next_overdue_book(queue, now)
except NoOverdueBooks:
    pass              # 이 문장이 실행되리라 예상함
else:
    assert False      # 이 문장은 결코 실행되지 않음

타임 머신
오만과 편견


In [13]:
# heapq 모듈의 벤치마크 성능

def heap_overdue_benchmark(count):
    def prepare():
        to_add = list(range(count))
        random.shuffle(to_add)
        return [], to_add

    def run(queue, to_add):
        for i in to_add:
            heappush(queue, i)
        while queue:
            heappop(queue)

    tests = timeit.repeat(
        setup='queue, to_add = prepare()',
        stmt=f'run(queue, to_add)',
        globals=locals(),
        repeat=100,
        number=1)

    return print_results(count, tests)

baseline = heap_overdue_benchmark(500)
for count in (1_000, 1_500, 2_000):
    comparison = heap_overdue_benchmark(count)
    print_delta(baseline, comparison)
    
# 결과 선형으로 증가한다, 원소수에 비례하여


원소 수:   500 걸린시간: 0.000560초

원소 수: 1,000 걸린시간: 0.001127초
데이터 크기  2.0배, 걸린 시간  2.0배

원소 수: 1,500 걸린시간: 0.001739초
데이터 크기  3.0배, 걸린 시간  3.1배

원소 수: 2,000 걸린시간: 0.002372초
데이터 크기  4.0배, 걸린 시간  4.2배


In [ ]:
# heapq 구현에 있어서 제시간에 반납된 책은 어떻게 처리해야할까?

# 해결 방법은 만기일 까지 우선순위 큐에서 책을 절대 제거 하지 않는 것이다.
# 만기일이 되면 정상 반납된 책이 우선순위 큐의 맨 앞에 있으므로 큐에서 연체된 책을 처리할 때 이미 반환된 책은 그냥 무시하면 된다.

@functools.total_ordering
class Book:
    def __init__(self, title, due_date):
        self.title = title
        self.due_date = due_date
        self.returned = False       # 새로운 필드

    def __lt__(self, other):
        return self.due_date < other.due_date

def next_overdue_book(queue, now):
    while queue:
        book = queue[0]
        if book.returned: # 이미 반환되었다면
            heappop(queue) # 큐에서 제거하고
            continue # 다시 우선순위큐 제거 과정 시작

        if book.due_date < now:
            heappop(queue)
            return book 

        break # 반납도 안되고, 연체된 책이 없는경우 종료

    raise NoOverdueBooks
    
def return_book(queue, book):
    book.returned = True
    


이 해법의 다넞ㅁ은 모든 책이 대출된 후 만기 이전에 반환된 경우

가장 빠른 만기일이 될 때까지는 힙의 크기가 최대 크기에서 줄어들지 않는다는 것이다.

heapq는 연산은 빨라지지만, 저장소 부가비용으로 메모리 사용량이 크게 늘어날 수 있다.

이런 단점에도 불구하고 튼튼한 시스템을 구축하려고 한다면 (최악의 경우 가정하고 계획을 세워야함)

1. 자연재해로 인해 도서관으로 가는 길이 막힌다던지
2. 대출된 모든 책이 연체될 수도 있다던지
...

이런 메모리 비용은 대출을 할 때 제약을 추가하여 해결 할 수도 있다.